# Get traffic sign features

In [ ]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [ ]:
import set_path

import json
import requests

import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame

import folium

import plot_utils

import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

In [ ]:
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

### Settings

In [ ]:
CRS = st.CRS

In [ ]:
# Input paths
# https://data.amsterdam.nl/datasets/AlrMVAnBaUNb-w/verkeersborden/?term=Verkeersborden
signs_file = '2022juni_verkeersborden_obv_beeldherkenning.csv'

### Functions

In [ ]:
def subset_data(df, df_loc):
    df_sub = gpd.sjoin(df, df_loc.set_geometry('buffer'), how='inner').reset_index()
    df_sub = df_sub.rename(columns = {'geometry_left': 'geometry'})
    df_sub = df_sub.set_geometry('geometry')
    return df_sub

## 1. Get traffic sign data

In [ ]:
df_signs_all = pd.read_csv(signs_file)
df_signs_all = GeoDataFrame(df_signs_all, geometry=gpd.points_from_xy(df_signs_all['x-coordinaat'], 
                                                                      df_signs_all['y-coordinaat']), crs=CRS)
df_signs_all = df_signs_all.drop(['x-coordinaat', 'y-coordinaat'], axis=1)

### Select crosswalk traffic signs

In [ ]:
#L02(F): Voetgangers oversteekplaats / zebrapad (blauw)
#J22(F): Voetgangersoversteekplaats met een zebrapad (vooraanduiding)  (rood)
#J23(F): U nadert een niet gemarkeerde voetgangers oversteekplaats

df_signs = df_signs_all[df_signs_all['rvv-modelnummer'].isin(['J22', 'J22F',
                                                              'L02', 'L02F', 'L02OB15F'])].reset_index(drop=True)
df_signs = df_signs.dropna(axis=1, how='all')  # remove columns without values

### Get mask of area

In [ ]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)
polygon = df_areas.to_crs(crs=CRS).unary_union

### Extract traffic sign information in polygon of area

In [ ]:
tl_data_in_area_mask = polygon.contains(df_signs['geometry'])
tl_data_in_area = df_signs.loc[tl_data_in_area_mask]

### Inspect data

In [ ]:
if tl_data_in_area.empty:
    print('No sidewalk data available in area')

tl_data_in_area

## 2. Check traffic signs data

### Plot traffic signs data on html map 

In [ ]:
# set True for satelite background, False for regular background
satelite = False

# Create Folium map
map = folium.Map(
    location=[52.389164, 4.908453], tiles=plot_utils.generate_map_params(satelite=satelite),
    min_zoom=10, max_zoom=25, zoom_start=15,
    zoom_control=True, control_scale=True, control=False
    )

# Add traffic sign features
feature_names = tl_data_in_area.columns.tolist()
feature_names.remove('geometry')
tooltip = plot_utils.gen_tooltip(feature_names, feature_names)
geo_j = folium.GeoJson(tl_data_in_area, tooltip=tooltip, style_function=lambda x: {"color": "orange", "weight": 10}).add_to(map)

map

### Save features as csv and html map

In [ ]:
# Save traffic sign crossing features as csv
tl_data_in_area.to_csv(cf.output_traffic_sign_crossing_features)